In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd

In [2]:
# lv0 = '../data'
# lv1s = os.listdir(lv0)
# for lv1 in lv1s:
#     lv2s = os.listdir('/'.join([lv0, lv1]))
#     asf_path = '%s/%s/%s.asf' % (lv0, lv1, lv1)
#     print('parsing %s' % asf_path)
#     motions = parse_amc('./nopose.amc')

In [3]:
def read_frame(frame, frames_dict):
    # parse joints
    for joint in frame:
        joint_info = joint.strip().split()
        joint_name = joint_info[0]
        joint_info.remove(joint_name)
        
        for i in range(len(joint_info)):
            coord = joint_info[i]
            frames_dict[joint_name + '_' + str(i)] = float(coord)
            
    return frames_dict

In [4]:
def parse_amc(filepath):
    with open(filepath) as f:
        content = f.read().splitlines()

    for idx, line in enumerate(content):
        if line == ':DEGREES':
            content = content[idx+1:]
            break
            
    # get idx of frames beginning
    frame_beginning = []
    for idx, line in enumerate(content):
        splitted_line = line.strip().split()
        if len(splitted_line) == 1:
            frame_beginning.append(idx)
    # parse frames
    all_frames = []
    for i in range(len(frame_beginning)-1):
        frame = content[frame_beginning[i]:frame_beginning[i+1]]
        frames_dict = {}
        frames_dict['frame_number'] = frame[0]
        # parse joints
        frames_dict = read_frame(frame[1:], frames_dict)
        all_frames.append(frames_dict)

    all_data = pd.DataFrame(all_frames)            
        
    return all_data

In [5]:
content = parse_amc('../data/05_01.amc')

In [6]:
content.head()

,frame_number,head_0,head_1,head_2,lclavicle_0,lclavicle_1,lfemur_0,lfemur_1,lfemur_2,lfingers_0,...,rwrist_0,thorax_0,thorax_1,thorax_2,upperback_0,upperback_1,upperback_2,upperneck_0,upperneck_1,upperneck_2
0,1,17.0477,-2.38560,1.88663,5.864150e-15,-2.067360e-14,-23.3432,-9.63608,-22.4650,7.12502,...,-19.8069,-0.989394,0.716381,-0.825246,-3.23848,1.409830,-1.715420,39.1252,-7.25169,6.39940
1,2,17.0567,-2.22129,1.86326,-2.246270e-14,2.544440e-14,-23.1047,-9.69850,-22.3475,7.12502,...,-19.0237,-0.594445,0.655129,-0.391631,-2.96777,1.285850,-1.278480,38.9078,-6.65582,6.03187
2,3,16.6771,-2.27540,1.85255,-1.316950e-14,-8.746530e-15,-22.1021,-10.00100,-22.3221,7.12502,...,-17.8653,-1.153680,0.699708,-0.708396,-3.16232,1.384640,-1.337120,38.5872,-6.75999,6.06970
3,4,16.8024,-2.13273,1.79075,5.466580e-16,1.828820e-14,-22.0129,-10.05590,-22.0734,7.12502,...,-16.7759,-0.288579,0.256131,-0.043233,-2.38539,0.492195,-0.841297,38.1986,-6.36379,5.80650
4,5,16.1301,-2.22479,1.84503,2.236330e-14,-1.073440e-14,-20.9293,-10.45660,-22.1086,7.12502,...,-15.7624,-0.556513,0.540161,-0.353442,-2.61813,1.068580,-0.881592,37.6700,-6.49567,5.94183


In [8]:
content.shape

(597, 63)